# Scenario 1 – Extended Simulation (Colab Ready)
이 노트북은 GRAPE 기반 Task Allocation 알고리즘을 확장하여 DFR Scenario 1 요구사항을 반영한 시뮬레이터입니다.
- 3D 공역 + 시간 슬롯 기반
- ETA, 연료 기반 우선권 반영
- 시각화를 포함하여 결과를 분석할 수 있도록 구성됨

In [ ]:
# 시각화 및 구조 준비
import matplotlib.pyplot as plt
import random
import math


In [ ]:
# TaskBlock, Agent, Airspace3D 정의
class TaskBlock:
    def __init__(self, tid, x, y, altitude, time_slot):
        self.tid = tid
        self.x = x
        self.y = y
        self.altitude = altitude
        self.time_slot = time_slot
        self.coalition = []
    def update_coalition(self, agent):
        if agent not in self.coalition:
            self.coalition.append(agent)
    def is_available(self, current_time):
        return self.time_slot == current_time

class Agent:
    def __init__(self, aid, start, goal, speed, fuel_level, eta):
        self.aid = aid
        self.start = start
        self.goal = goal
        self.position = start
        self.speed = speed
        self.fuel_level = fuel_level
        self.eta = eta
        self.assigned_task = None

    def evaluate_tasks(self, tasks, current_time):
        best_utility = -float('inf')
        best_task = None
        for task in tasks:
            if not task.is_available(current_time):
                continue
            d_ij = self.distance((task.x, task.y))
            p_collision = len(task.coalition) / 10
            wait_time = abs(task.time_slot - current_time)
            priority = self.compute_priority(task)
            utility = (
                1.0 / (d_ij + 1e-5) +
                (1 - p_collision) +
                1.0 / (wait_time + 1e-5) -
                len(task.coalition) +
                priority
            )
            if utility > best_utility:
                best_utility = utility
                best_task = task
        return best_task

    def compute_priority(self, task):
        return max(0.0, (self.fuel_level / 100) + (1.0 - abs(task.time_slot - self.eta) / 10))

    def distance(self, pos):
        dx = self.position[0] - pos[0]
        dy = self.position[1] - pos[1]
        return math.sqrt(dx**2 + dy**2)

class Airspace3D:
    def __init__(self, width, height, altitudes, time_slots):
        self.tasks = []
        tid = 0
        for t in time_slots:
            for z in altitudes:
                for x in range(width):
                    for y in range(height):
                        self.tasks.append(TaskBlock(tid, x, y, z, t))
                        tid += 1

In [ ]:
# 시뮬레이션 실행 및 시각화
def run_simulation():
    airspace = Airspace3D(width=3, height=3, altitudes=[100,200], time_slots=range(5))
    agents = [
        Agent(0, (0,0), (2,2), 1, 80, 2),
        Agent(1, (0,2), (2,0), 1, 60, 3)
    ]

    for t in range(5):
        for task in airspace.tasks:
            task.coalition = []
        for agent in agents:
            task = agent.evaluate_tasks(airspace.tasks, t)
            if task:
                task.update_coalition(agent)
                agent.assigned_task = task
        # 시각화
        plt.clf()
        for agent in agents:
            if agent.assigned_task:
                plt.plot([agent.position[0], agent.assigned_task.x], [agent.position[1], agent.assigned_task.y], 'k--')
                plt.scatter(agent.position[0], agent.position[1], c='blue')
                plt.scatter(agent.assigned_task.x, agent.assigned_task.y, c='red')
        plt.title(f"Step {t}")
        plt.xlim(-1, 4)
        plt.ylim(-1, 4)
        plt.pause(0.5)
    plt.show()

run_simulation()